# Euler project

$$
\partial_t w + \partial_x {F(w)} =  0
$$
exemple $F(w)=w^2/2$.



$$
w^{n+1}_i  = w^n_i -  \frac {\delta_t}{\delta_x} [F(w^n_{i+\frac 12}) - F(w^n_{i- \frac 12}) ]
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Introduction généraliste

* Léo : viscosité  spaciale: diffuser en fonction de la forme de la fonction. Ex fort graident
    * local ou global
    * on regarde les fonctions une à une. 
    * Selon les régime plysique, tu veux plus difuser les choses les plus rapides. Pas nécessaire en GD: il y a déjà de la viscosité physique.  Alors qu'en élément fini on est obligé de faire les 2


* Euler: viscosité physique. Couplé. Qui ne dépend que des interaction entre les variables. quel phénomène est le plus rapide, le plus fort. 
    * On s'interdit de regarder globallement.
    * On diffuse en rho, rhoV, E. 






### EDP

On se donne une fonction $F:\mathbb R^n \to \mathbb R^n $. 

On cherche une fonction $(x,t)\to w(x,t)\in \mathbb R^n$ qui satisfait à l'EDP suivante:
$$ \tag{EDP}
 \partial_t w = - \partial_x F(w)
$$
L'évolution en temps de $w$ est donnée par une dérivée spaciale de $F(w)$ que l'on appelle naturellement le flux.  

### Schéma

* On a une discrétisation temporelle $t_0,t_1,...$.  
* On a une discrétisation spaciale $x_0,x_1,...$.  
* On condisère aussi les points spaciaux du milieu. $x_{i+\frac 12}=(x_i + x_{i+1})/2$

On note
* $w^n_i = w(x_i,t_n)$
* $w^n_{i+\frac 12} = w(x_{i+\frac 12},t_n)$

L'équation $(EDP)$ s'écrit alors: 
$$
\frac 1{\delta_t} [w^{n+1}_i - w^n_i ] = - \frac 1{\delta_x} [F(w^n_{i+\frac 12}) - F(w^n_{i- \frac 12}) ]
$$
ou encore
$$
w^{n+1}_i  = w^n_i -  \frac {\delta_t}{\delta_x} [F(w^n_{i+\frac 12}) - F(w^n_{i- \frac 12}) ]
$$
Ce schémat nous permettra d'évoluer en temps. 








## Trouver une fonction interpolante 

***NB:*** Par la suite, et jusqu'à la nouvelle intervention du temps, on se fixe un temps $t_n$. Et abrège $v_i^n,w_i^n$ par $v_i,w_i$. 







### Approximation du Flux

Il nous faut maintenant une fonction qui nous permette de rester toujours sur le même maillage. 
On suppose l'existance d'une fonction $F_?$ à deux variable telle que
$$
F_?(w_i,w_{i+1}) \approx  F(w_{i+\frac 12})
$$
$F_?$ est appelé le flux numérique. Le choix de cette fonction est très important. Tout le travail qui suit consite à trouver un bon candidat. Un mauvais choix crée des problèmes de stabilité: les erreur  se propagent quand on applique successivement le schéma numérique.

***Variante:*** On peut imaginer une fonction à plus de variable:
$$
F_?(w_{i-1},w_i,w_{i+1},w_{i+2}) \approx  F(w_{i+\frac 12})
$$

### ou approximation d'une différence de Flux

 
Le terme droit du schéma numérique est:
$$
 w_i -  \frac {\delta_t}{\delta_x} [F(w_{i+\frac 12})-F(w_{i-\frac 12})]
$$

Pour garder une maillage constant, une autre solution est d'essayer de trouver directement une fonction $F\!F_?$ telle que
$$
F\!F_?(w_{i-1},w_i,w_{i+1}) \approx F(w_{i+\frac 12})-F(w_{i-\frac 12})
$$
***variante:*** Et pourquoi pas une fonction à plus de variables
$$
F\!F_?(w_{i-2},w_{i-1},w_i,w_{i+1},w_{i+2}) \approx F(w_{i+\frac 12})-F(w_{i-\frac 12})
$$



Anticipons: dans nos expérimentation, l'approximation directe de la différence de flux par des réseaux de neurones donne des résultats moins bons en terme de stabilité.  

### Mauvais candidat naturel

Un premier candidat naturel pour le flux numérique est de prendre la moyenne:
$$
F_?(w_i,w_{i+1}) = \frac{F(w_i)+F(w_{i+1})}{ 2} 
$$
Mais il est très instable dans le temps. 


Pour améliorer sa stabilité, une technique classique consiste à jouter un terme à la moyenne:
$$
F_?(w_i,w_{i+1}) = \frac{F(w_i)+F(w_{i+1})}{ 2}  + A(w_i,w_{i+1})(w_{i+1}-w_i)
$$
La fonction $A$ est souvent prise bilinéaire. 

### Le flux HLL

Pour simplifier, notons $F_i=F(w_i)$. Le Flux HLL  est donné par la formule:
$$
F_{HLL} (w_i,w_{i+1})=
\begin{cases}
F_{i} & \text{si } \lambda_{i} \geq 0 \\
\frac{\lambda_{i+1} F_{i} - \lambda_{i} F_{i+1} }{\lambda_{i+1}-\lambda_{i}} +
\frac{\lambda_{i+1}\lambda_{i} (w_{i+1} -  w_{i}) }{\lambda_{i+1}-\lambda_{i}}
 & \text{si } \lambda_i<0< \lambda_{i+1} \\
 F_{i+1} &\text{si } \lambda_{i+1}\leq 0 
\\
\end{cases}
$$
où $\lambda_i=...$.


### Notre candidat


C'est un très bon candidat. Notre travaille consiste à l'améliorer avec un terme additif:
$$
F_{\theta} (w_i,w_{i+1})=F_{HLL} (w_i,w_{i+1})+ A_\theta(w_i,w_{i+1})
$$
et nous testerons aussi des solutions à 4 variables:
$$
F_{\theta} (w_{i-1},w_i,w_{i+1},w_{i+2})=F_{HLL} (w_i,w_{i+1})+ A_\theta(w_{i-1},w_i,w_{i+1},w_{i+2})
$$
où plus (6,8,...)

## Apprentissage supervisé. 

Grace à une grille très fine, nous avons réussit à calculer des séries de $v_i^n$ très précis. Nous allons pouvoir entrainer des réseaux de neurones.  









### Baseline

Un réseau de neurone $G_\theta(w_{i-1},w_i,w_{i+1})$ qui est cencé directement reproduire $v_i$. On ne le structure pas du tout. La loss associée sera diretement
$$
\mathtt{Loss}^G_i = G(w_{i-1},w_i,w_{i+1}) - v_i
$$
puis
$$
\mathtt{Loss}^G= \sum_i \mathtt{Loss}^G_i
$$



### Le réseau différence

On se donne un réseau $F_\theta(w_i,w_{i+1})$. On prend comme première loss:
$$
\mathtt{lossDiff}_i = \frac{F_\theta(w_i,w_{i+1})- F_\theta(w_{i-1},w_{i})}{\delta_x}  - v_i
$$
puis
$$
\mathtt{lossDiff} = \sum_i \mathtt{lossDiff}_i
$$
Mais avec  cette seule loss, l'apprentissage ne pourra pas converger: en ajoutant une constante à $F_\theta$ on ne change pas la loss.  

On peut alors ajouter un terme de pénalisation par exemple:
$$
\mathtt{loss}_{\alpha} = \mathtt{lossDiff}+ \alpha \sum w^2
$$








### Une seconde contraine

Mais on peut aussi penser à une seconde contrainte:

On aimerait aussi que notre réseau satisface cette contrainte: 
$$\tag{C2}
F_\theta(w_i,w_i) \approx  F(w_i)
$$
Cette contrainte est naturelle quand on se souvient que
$$
F_\theta(w(x_i),w(x_{i+1})) \text{ doit approcher } F\circ w (\frac {x_i+x_{i+1}}2)
$$
Quand le pas du maillage tend vers 0, on obtient cette seconde contrainte. Ainsi:

$$
\mathtt{loss}^{C2}=\mathtt{lossDiff}  +  \sum_i F_\theta(w_i,w_i)-F(w_i)
$$
Remarquons que cette seconde condition ne lève par complètement l'intermination: en ajoutant à $F_\theta$ une fonction $H(w_i,w_{i+1})$ qui s'annule sur la diagonale, la loss ne change pas. Il faut surement un peu pénaliser aussi. 


### Réseau addifif

C'est le réseau le plus structuré. C'est un peu l'idée des resNet: le réseau de neurone ajoute un terme à une quantité déjà bonne: 

On  considére un réseau de neurone $A_\theta$ puis
$$
F^A_\theta(w_i,w_{i+1}) = \frac{F(w_i)+F(w_{i+1})}{ 2}  + A_\theta(w_i,w_{i+1})(w_{i+1}-w_i)
$$
Remarquons que le solution $(C2)$ est automatiquement satisfaite par ce réseau. 

Ensuite on fait comme d'habitude: 
$$
\mathtt{lossDiff}^A_i =  F^A_\theta(w_i,w_{i+1}) -F^A_\theta(w_{i-1},w_{i}) - v_i  
$$
puis on pénalise: 
$$
\mathtt{loss}^A_{\alpha} = \mathtt{lossDiff}^A+ \alpha \sum w^2
$$


### Plus de points pour interpoler ? 

Pour faire une interpolation précise il faut souvent considérer les voisins plus lointains. On pourait donc prendre
$$ 
F_\theta(w_{i-1},w_i,w_{i+1},w_{i+2}) \approx F( w_{i+\frac 12})
$$
Ce qui conduirait à une différence
$$
F\!F_\theta (w_{i-2},w_{i-1},w_i,w_{i+1},w_{i+2}) \approx v_i
$$
Et comme baseline un simple réseau non structuré
$$
G_\theta (w_{i-2},w_{i-1},w_i,w_{i+1},w_{i+2}) \approx v_i
$$
Ou en poussant encore plus loin de bouchon: on pourrait tester une estimation globale en espace de type Vnet. 
$$
Vnet_\theta(w_.) \approx v_.
$$
sachant que le Vnet est une réseau de convolution, donc il peut glisser sur des données de taille arbitraire. Il produit simplement plusieurs output d'un coup. 

## Notre problème en particulier

### l'EDP


* $\rho(x,t)$ : une densité de gaz neutre. 
* $E$ : Energie totale du gaz
* $V$ : vitesse moyenne du gaz

Les inconnues sont 
* $\rho$
* $V$ (ou $\rho V$)
* $p$ (ou $E$).

Le système d'équations est:
$$
\partial_t \rho + \partial_x (\rho V) = 0 
$$
$$
\partial_t \rho V + \partial_x (\rho V^2 +p)   =0
$$
$$
\partial_t E +\partial_x (EV + pV)  =0
$$
$p$ et $E$ sont équivalente par la formule suivante:
$$
E = \frac 12 \rho V^2+ \frac 1{\gamma - 1 } p
$$

$\gamma$ est un paramètre physique. Entre $1$ et $3$. (pour l'instant fixe). 



***Attention:*** Par rapport au doc de référence, chez nous 
$E$ correspond à $\rho E$ chez eux.

### Vectorisation

On note
$$
w(x,t)=[\rho(x,t),\rho V(x,t), E(x,t)]
$$
$$
v(x,t) = \partial_t w(x,t)
$$
On considère ensuite:
$$
F( w ) = -[\rho V, \rho V^2+p, EV+p V]
$$
(La fonction $F$ peut facilement être explicitée).  

l'EDP se réécrit alors: 
$$
v =  \partial_x F( w)
$$
et on rentre dans le cadre général décrit précédemment. 



***Remarque:*** En pratique, pour nourir le machine learning on utiliser un vecteur "augmenté" (data augmentation)
$$
\bar w(x)=[\rho(x),\rho V(x), E(x), V(x), p(x),...]
$$

## Intervention du temps

Le supervisé est la pour la consistance

Le renforcement est là pour la stabilité, puisqu'il regarde tous les temps. 

Côté multi-agent. 

    s'r= env(s,a) 

ici : plein de petit 

    s = w_l,w_r
    a = F_theta #1 seule politique

    S=s_1,...,s_100
    A=a_1,...,a_100

    S',r=env(S,A)


Problème complétement coopératif: la même politique pour chaque agent. L'action est "locale". 







### Moyen d'assurer la stabilité

...


Une fonction de entropie  $H(w_i)$. 

tu construit une second réseau ne neurone

***Théoreme:*** Si il existe un $Q_\theta$ vérifiant ceci
$$
[H(w_i^{n+1})-H(w_i^{n})] / \delta_i  + [Q_\theta(w_i^n ,w_{i+1}^n )- Q_\theta(w_{i-1}^n ,w_{i}^n )] / \delta_x \leq 0
$$
Alors c'est stable. 


$$
H(w_i^{n+1}) = ....
$$

Et le $w^{n+1}$ c'est celui obtenu avec $F_\theta$. Tu calcul un 
$$
H(w^*)= 
$$
pour que l'inégalité soit accepté et il faut que 

On construit les 2 réseaux.

### La solution RNN

A partir de maintenant pour simplifier, nous allons parler uniquement de $F\! F_\theta$ sachant de la meilleurs technique (sans doute) c'est de l'exprimer comme une différence de 2 réseaux de neurones. 

Et nous simplifierons l'équation locale
$$
v_i = F\! F_\theta(w_{i-1},w_i,w_{i+1}) 
$$
Par la notation  "globale" en espace (mais ce n'est qu'une notation, nous pensons toujours que $F\! F$  agit semi-localement
$$
v = F\! F_\theta(w) 
$$
... A suivre




### Exagérer les problèmes pour mieux les minimser. 

Il faut que les  erreurs sur la sortie de $F_\theta$ se ressentent fortement sur la loss.



